#Create a Search Job and Get SID from Splunk

In [ ]:
import requests
from xml.dom import minidom

# Set the URL and authentication credentials
url = "http://03d77e6c06f1.ngrok.app/services/search/jobs/"
username = "sc_admin"
password = "Splunk123"

# Create a dictionary of the search parameters
search_params = {
    "search": "search index=main earliest=-7d"
}

# Make the request to the Splunk API
response = requests.post(url, auth=(username, password), data=search_params)

# Check the response status code
if response.status_code == 201:
    # The request was successful
    print(response.text)
else:
    # The request failed
    print("Error: {}".format(response.status_code))

dom = minidom.parseString(response.text)
# Get the SID element
sid_element = dom.getElementsByTagName("sid")[0]
sid = sid_element.firstChild.nodeValue
print(sid)

# Get Results for the SID

In [ ]:

import json
results_url = "http://03d77e6c06f1.ngrok.app/services/search/jobs/{}/results".format(sid)
response = requests.get(results_url, auth=(username, password),params={"output_mode": "json_rows"})
import pandas as pd
json_data = json.loads(response.text)
# json_data

In [ ]:
def extract_data(json_data):
    rows = json_data['rows']
    data = []
    for row in rows:
        raw_data = row[3]
        parsed_data = json.loads(raw_data)
        data.append({
            'timestamp': parsed_data['timestamp'],
            'user_id': parsed_data['User_ID'],
            'amount': parsed_data['amount'],
            'action': parsed_data['action'],
            'location': parsed_data['location'],
            'is_fraud': parsed_data['is_fraud']
        })
    return data


In [ ]:
# Extract the data from the JSON
data = extract_data(json_data)

# Create a pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)
print(df.info())



### Detect Anomaly

In [ ]:
import pandas as pd
from google.cloud import bigquery
import json
import requests
from termcolor import colored

# Create a BigQuery client
client = bigquery.Client()

# Create the query
query_template = """
SELECT
  user_id, ts, amount, lower_bound, upper_bound, anomaly_probability, is_anomaly AS anomaly
FROM
  ML.DETECT_ANOMALIES(MODEL `finance.arima_plus_timeseries_banking`, STRUCT(0.9 AS anomaly_prob_threshold),
  (
    SELECT @user_id AS user_id, TIMESTAMP(@timestamp) AS ts, @amount AS amount
  ))
"""


# Iterate over DataFrame rows and execute the query for each row
for index, row in df.iterrows():
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("user_id", "INT64", row["user_id"]),
            bigquery.ScalarQueryParameter("timestamp", "STRING", row["timestamp"]),
            bigquery.ScalarQueryParameter("amount", "FLOAT64", row["amount"])
        ]
    )
    query = query_template
    query_job = client.query(query, job_config=job_config)
    results = query_job.result().to_dataframe()
    json_records = results.to_json(orient='records')
    json_records = json.loads(json_records)
    # If json_records has anomaly_probability is more than 0.9 send it to Splunk
    data=json.dumps({"event": json_records[0], "sourcetype": "_json","index":"anomalies"})
    print(data)
    if results.iloc[0]['anomaly_probability'] >= 0.90:
        response = requests.post("http://492bb2284e5b.ngrok.app/services/collector/event", headers={"Authorization": "Splunk 292cdf32-0581-48f6-ae52-e3d0277b098d"}, data=json.dumps({"event": json_records[0], "sourcetype": "_json","index":"anomalies"}))
        # Print Anomaly in Red
        print(json_records[0])







